In [1]:
from itertools import islice
with open("wikicorpus_01") as myfile:
    head = list(islice(myfile, 5651))

In [2]:
print(head[:9])

['Néstor_Gabriel_Martinena néstor_gabriel_martinena NP00000 0\n', '( ( Fpa 0\n', 'La el DA0FS0 0\n', 'Plata plata NP00000 10501473\n', ', , Fc 0\n', 'Buenos_Aires buenos_aires NP00000 0\n', ', , Fc 0\n', 'Argentina argentina NP00000 02035234\n', ', , Fc 0\n']


In [3]:
sent = []
for s in head:
    sent.append(s.split())

In [4]:
sent[:5]

[['Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'],
 ['(', '(', 'Fpa', '0'],
 ['La', 'el', 'DA0FS0', '0'],
 ['Plata', 'plata', 'NP00000', '10501473'],
 [',', ',', 'Fc', '0']]

In [5]:
words = []
sentences = []
for s in sent:
    if len(s) == 4 and s[0] not in {'(', ')', ','} and '[' not in s[1]:
        words.append((s[1], s[2], s[3]))
        #if 'endofarticle' in s[1]:
        #    sentences.append(words)
        #    words = []

In [6]:
count = dict()
for fst,snd,trd in words:
    if fst not in count:
        count[fst] = 1
    else:
        count[fst] += 1

In [7]:
count['entrar']

2

In [8]:
words[:4]

[('néstor_gabriel_martinena', 'NP00000', '0'),
 ('el', 'DA0FS0', '0'),
 ('plata', 'NP00000', '10501473'),
 ('buenos_aires', 'NP00000', '0')]

In [9]:
aux = []
check = []
for fst,snd,trd in words:
    aux.append((fst,snd, trd))
    if fst == '.':
        check.append(aux)
        aux = []

In [10]:
check[:4]

[[('néstor_gabriel_martinena', 'NP00000', '0'),
  ('el', 'DA0FS0', '0'),
  ('plata', 'NP00000', '10501473'),
  ('buenos_aires', 'NP00000', '0'),
  ('argentina', 'NP00000', '02035234'),
  ('ser', 'VSIP3S0', '01775973'),
  ('uno', 'DI0MS0', '0'),
  ('futbolista', 'NCCS000', '0'),
  ('argentino', 'AQ0MS0', '00278090'),
  ('.', 'Fp', '0')],
 [('jugar', 'VMM02S0', '00727813'),
  ('de', 'SPS00', '0'),
  ('delantero', 'NCMS000', '00466114'),
  ('y', 'CC', '0'),
  ('su', 'DP3CS0', '0'),
  ('equipo', 'NCMS000', '06093198'),
  ('actual', 'AQ0CS0', '01667781'),
  ('ser', 'VSIP3S0', '01775973'),
  ('gimnasia', 'NP00000', '00275488'),
  ('y', 'CC', '0'),
  ('esgrima_la_plata_de_la_primera_división_de_argentina', 'NP00000', '0'),
  ('.', 'Fp', '0')],
 [('trayectoria', 'NP00000', '06709272'), ('.', 'Fp', '0')],
 [('debutar', 'VMIS3S0', '01176337'),
  ('en', 'SPS00', '0'),
  ('1', 'AO0FS0', '02098880'),
  ('el', 'DA0MS0', '0'),
  ('frente_a', 'SPS00', '0'),
  ('river_plate', 'NP00000', '0'),
  ('entra

In [11]:
from collections import defaultdict

In [12]:
dicc_backward = defaultdict(list)
dicc_forward = defaultdict(list)
for s in check:
    for i,w in enumerate(s):
        if i != 0:
            dicc_backward[w[0]].append((s[i-1][0], s[i-1][1]))
        else:
            dicc_backward[w[0]].append(('[W.Start]', '[T.Start]'))
        if i != len(s)-1:
            dicc_forward[w[0]].append((s[i+1][0], s[i+1][1]))
        else:
            dicc_forward[w[0]].append(('[W.End]', '[T.End]'))

In [13]:
dicc_backward['entrar']

[('river_plate', 'NP00000'), ('el', 'DA0MS0')]

In [14]:
dicc_forward['entrar']

[('desde', 'SPS00'), ('a', 'SPS00')]

In [15]:
dicc = dict()
vec = dict()
for fst,snd,trd in words:
    features = defaultdict(int)
    pos = 'PoS__' + snd
    if fst not in dicc and fst not in {'.', ',', ':', ';'} and fst.isalpha() and count[fst] > 1 and trd != 0:
        dicc[fst] = features
        vec[fst] = trd
        features[pos] = 1
        for i in dicc_backward[fst]:
            features[i[0]+"-1"] += 1
            features[i[1]+"-1"] += 1
        for i in dicc_forward[fst]:
            features[i[0]+"+1"] += 1
            features[i[1]+"+1"] += 1
        
    elif fst in dicc:
        if trd not in vec[fst]:
            dicc[fst] = features
            vec[fst] = trd
            for i in dicc_backward[fst]:
                features[i[0]+"-1"] += 1
                features[i[1]+"-1"] += 1
            for i in dicc_forward[fst]:
                features[i[0]+"+1"] += 1
                features[i[1]+"+1"] += 1
        has_it = dicc[fst]
        if pos in has_it:
            has_it[pos] += 1
        else:
            has_it[pos] = 1

In [16]:
dicc['con']

defaultdict(int,
            {'1-1': 1,
             '35+1': 1,
             'AQ0CS0-1': 3,
             'AQ0FP0-1': 1,
             'AQ0FS0-1': 3,
             'AQ0MP0-1': 1,
             'DA0FS0+1': 6,
             'DA0MP0+1': 2,
             'DA0MS0+1': 3,
             'DA0NS0+1': 1,
             'DI0FS0+1': 3,
             'DI0MP0+1': 1,
             'DI0MS0+1': 2,
             'DP3CS0+1': 3,
             'NCFP000+1': 2,
             'NCFP000-1': 2,
             'NCFS000+1': 1,
             'NCFS000-1': 6,
             'NCMN000+1': 1,
             'NCMP000+1': 3,
             'NCMP000-1': 1,
             'NCMS000+1': 3,
             'NCMS000-1': 5,
             'NP00000+1': 8,
             'NP00000-1': 6,
             'PR0CS000+1': 1,
             'PoS__SPS00': 41,
             'RG-1': 1,
             'VMII1S0-1': 1,
             'VMIP1S0-1': 1,
             'VMIP3P0-1': 1,
             'VMIP3S0-1': 5,
             'VMN0000-1': 1,
             'VMP00SF-1': 1,
             'VSIP3S0-

In [18]:
matrix = []
vector = []
final_check = []
for key, value in dicc.items():    #preparo la lista de dicts y la lista de chequeo para recuperar el nombre al final
    final_check.append(key)
    matrix.append(value)
for key, value in vec.items():       #vector de synsets
    vector.append(int(value))

In [20]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
X = v.fit_transform(matrix)

In [21]:
mejor_k = X.shape

In [22]:
k_best = mejor_k[1]*0.7

In [24]:
##########################################Supervisada, clase: synset ##################################################

In [25]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [26]:
Y = SelectKBest(chi2, k=int(k_best)).fit_transform(X, vector)

In [27]:
#Normalización
import sklearn
Y = sklearn.preprocessing.normalize(Y)

In [28]:
#Clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=15)
km.fit(Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=15, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [29]:
labels = km.predict(Y)

In [30]:
clusters = defaultdict(set)
for i, label in enumerate(labels):
    clusters[label].add(final_check[i])
print(clusters)

defaultdict(<class 'set'>, {13: {'directo', 'crear', 'ambos', 'participar', 'morir', 'transformar', 'destacar', 'aceptar', 'vivir', 'apreciar', 'mediocampista', 'utilizar', 'el', 'tipo', 'filmar', 'realizar', 'numeroso', 'vengar', 'representar', 'asesinar', 'tomar', 'uno', 'otro', 'imitar', 'español', 'publicar', 'hacer', 'mi', 'su', 'varios', 'moderno'}, 2: {'revista', 'importancia', 'canción', 'cual', 'localidad', 'forma', 'acción', 'masa', 'organización', 'aparición', 'banda', 'falta', 'traducción', 'píldora', 'escena', 'información', 'hora', 'ilustración', 'versión', 'lengua', 'fraternidad', 'bomba', 'ciudad', 'voz', 'provincia', 'plata', 'comarca', 'parroquia', 'sociedad', 'razón', 'oportunidad'}, 7: {'la', 'jökulsárlón', 'cross', 'centralita', 'bond', 'grecia', 'américa', 'sloan', 'castelao', 'ecu', 'youtube', 'nightwish', 'medicina', 'holopainen', 'enlaces', 'francia', 'descripción', 'endofarticle', 'referencias', 'mcavoy', 'españa', 'bibliografía', 'nadie', 'wanted', 'islandia'